In [0]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

In [0]:
!pip install OpenNMT-py

In [0]:
!onmt_preprocess -train_src OpenNMT-py/data/src-train.txt -train_tgt OpenNMT-py/data/tgt-train.txt -valid_src OpenNMT-py/data/src-val.txt -valid_tgt OpenNMT-py/data/tgt-val.txt -save_data OpenNMT-py/data/data -src_vocab_size 10000 -tgt_vocab_size 10000

In [3]:
import torch
import torch.nn as nn

import onmt
import onmt.inputters
import onmt.modules
import onmt.utils

"""
We begin by loading in the vocabulary for the model of interest. 
This will let us check vocab size and to get the special ids for padding
"""

vocab_fields = torch.load("OpenNMT-py/data/data.vocab.pt") #vocab field 만들기

src_text_field = vocab_fields["src"].base_field #src field 만들기
src_vocab = src_text_field.vocab
src_padding = src_vocab.stoi[src_text_field.pad_token] #Padding

tgt_text_field = vocab_fields['tgt'].base_field #tgt field 만들기
tgt_vocab = tgt_text_field.vocab
tgt_padding = tgt_vocab.stoi[tgt_text_field.pad_token] #Padding

"""
Next we specify the core model itself. 
Here we will build a small model with an encoder and an attention based input feeding decoder. 
Both models will be RNNs and the encoder will be bidirectional
"""

emb_size = 100
rnn_size = 500
# Specify the core model.

encoder_embeddings = onmt.modules.Embeddings(emb_size, len(src_vocab),
                                             word_padding_idx=src_padding) #src 임베딩

encoder = onmt.encoders.RNNEncoder(hidden_size=rnn_size, num_layers=1,
                                   rnn_type="LSTM", bidirectional=True,
                                   embeddings=encoder_embeddings) #인코더 

decoder_embeddings = onmt.modules.Embeddings(emb_size, len(tgt_vocab),
                                             word_padding_idx=tgt_padding) #tgt 임베딩

decoder = onmt.decoders.decoder.InputFeedRNNDecoder(
    hidden_size=rnn_size, num_layers=1, bidirectional_encoder=True, 
    rnn_type="LSTM", embeddings=decoder_embeddings) #Decoder 

device = "cuda" if torch.cuda.is_available() else "cpu" #Device 설정 

model = onmt.models.model.NMTModel(encoder, decoder) #모델 생성 
model.to(device)#디바이스 올리기 

# Specify the tgt word generator and loss computation module
model.generator = nn.Sequential(
    nn.Linear(rnn_size, len(tgt_vocab)),
    nn.LogSoftmax(dim=-1)).to(device) #제너레이터 부분 

loss = onmt.utils.loss.NMTLossCompute(
    criterion=nn.NLLLoss(ignore_index=tgt_padding, reduction="sum"),
    generator=model.generator) #Loss 설정

"""
Now we set up the optimizer. 
Our wrapper around a core torch optim class handles learning rate updates and gradient normalization automatically.
"""
lr = 1
torch_optimizer = torch.optim.SGD(model.parameters(), lr=lr) #옵티마이져
optim = onmt.utils.optimizers.Optimizer(
    torch_optimizer, learning_rate=lr, max_grad_norm=2) #옵티마이저

"""
Now we load the data from disk with the associated vocab fields. 
To iterate through the data itself we use a wrapper around a torchtext iterator class. 
We specify one for both the training and test data.
"""

# Load some data
from itertools import chain

train_data_file = "OpenNMT-py/data/data.train.0.pt" #학습데이터
valid_data_file = "OpenNMT-py/data/data.valid.0.pt" #검증데이터 

train_iter = onmt.inputters.inputter.DatasetLazyIter(dataset_paths=[train_data_file],
                                                     fields=vocab_fields,
                                                     batch_size=50,
                                                     batch_size_multiple=1,
                                                     batch_size_fn=None,
                                                     device=device,
                                                     is_train=True,
                                                     repeat=True,
                                                     pool_factor=True#기존 소스 코드 변경
                                                     ) #iterator 만들기

valid_iter = onmt.inputters.inputter.DatasetLazyIter(dataset_paths=[valid_data_file],
                                                     fields=vocab_fields,
                                                     batch_size=10,
                                                     batch_size_multiple=1,
                                                     batch_size_fn=None,
                                                     device=device,
                                                     is_train=False,
                                                     repeat=False,
                                                     pool_factor=True
                                                     )

"""
Finally we train. Keeping track of the output requires a report manager.
"""

report_manager = onmt.utils.ReportMgr(
    report_every=50, start_time=None, tensorboard_writer=None) #리포트 매니저

trainer = onmt.Trainer(model=model,
                       train_loss=loss,
                       valid_loss=loss,
                       optim=optim,
                       report_manager=report_manager) #트레이너 생성 

trainer.train(train_iter=train_iter,
              train_steps=400,
              valid_iter=valid_iter,
              valid_steps=200) #실제 학습진행




In [0]:
import onmt.translate

src_reader = onmt.inputters.str2reader["text"]
tgt_reader = onmt.inputters.str2reader["text"]

scorer = onmt.translate.GNMTGlobalScorer(alpha=0.7, 
                                         beta=0., 
                                         length_penalty="avg", 
                                         coverage_penalty="none")

gpu = 0 if torch.cuda.is_available() else -1

translator = onmt.translate.Translator(model=model, 
                                       fields=vocab_fields, 
                                       src_reader=src_reader, 
                                       tgt_reader=tgt_reader, 
                                       global_scorer=scorer,
                                       beam_size=1,
                                       gpu=gpu) #트랜스레이터 생성 

builder = onmt.translate.TranslationBuilder(data=torch.load(valid_data_file), 
                                            fields=vocab_fields) #빌더 생성 

for batch in valid_iter:
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=[src_vocab],
        attn_debug=False) #번역진행
    
    translations = builder.from_batch(trans_batch)#배치만큼
    for trans in translations:
        print(trans.log(0))